## 1. Importações e Configuração

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from sqlalchemy import create_engine, text
import warnings
warnings.filterwarnings('ignore')

print("Bibliotecas importadas com sucesso!")

Bibliotecas importadas com sucesso!


### Configuração de Conexão com Banco de Dados

In [2]:
# Configuração do banco de dados PostgreSQL
DB_HOST = "localhost"
DB_PORT = "5433"
DB_NAME = "brasileirao"
DB_USER = "postgres"
DB_PASSWORD = "postgres"

# String de conexão
CONNECTION_STRING = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Criar engine SQLAlchemy
engine = create_engine(CONNECTION_STRING, pool_size=10, max_overflow=20, echo=False)

print("Configuração do banco definida")
print(f"Host: {DB_HOST}:{DB_PORT}")
print(f"Database: {DB_NAME}")

Configuração do banco definida
Host: localhost:5433
Database: brasileirao


### Função de Transformação de Dados

### Documentação: Limpeza e Tratamento de Dados

**Problemas identificados nos dados RAW:**

1. **Valores nulos em colunas de partidas:**
   - `formacao_mandante` e `formacao_visitante`: 56.6% nulos
   - `tecnico_mandante` e `tecnico_visitante`: 52.5% nulos
   - **Solução:** Colunas removidas (dados insuficientes para análise)

2. **Strings com espaços extras:**
   - Encontrados em nomes de times, jogadores, clubes
   - **Solução:** Aplicado `.strip()` em todas colunas do tipo string

3. **Percentuais como strings:**
   - `posse_de_bola` e `precisao_passes` com valores "45%", "87.5%"
   - **Solução:** Removido símbolo "%" e convertido para numérico

4. **Valores nulos em estatísticas:**
   - ~23.808 valores nulos distribuídos em estatísticas
   - **Solução:** 
     - Contadores (chutes, passes, faltas) preenchidos com 0
     - Percentuais (posse, precisão) preenchidos com média da coluna

5. **Inconsistências em agregações:**
   - Partidas sem estatísticas detalhadas (dados antigos)
   - **Solução:** LEFT JOIN mantém todas partidas, nulos tratados posteriormente

**Resultado:**
- Dados normalizados e prontos para análise
- Big Table desnormalizada facilitando queries
- Integridade referencial mantida entre partidas, gols e cartões

In [ ]:
def transformar_dados_silver(df_partidas, df_gols, df_cartoes, df_estatisticas):
    """
    Pipeline completo de transformação para criar a Big Table Silver.
    Combina os 4 DataFrames RAW em uma única tabela desnormalizada.
    """
    print("\n" + "="*70)
    print("INICIANDO TRANSFORMAÇÃO RAW → SILVER")
    print("="*70)
    
    #  1. LIMPEZA E NORMALIZAÇÃO 
    print("\n[1/5] Limpeza e normalização de dados...")
    
    # Função auxiliar para normalizar strings
    def normalize_string(s):
        if isinstance(s, str):
            return s.strip()
        return s
    
    # Normalizar strings em todas as tabelas
    for col in df_partidas.select_dtypes(include=['object']).columns:
        df_partidas[col] = df_partidas[col].apply(normalize_string)
    
    for col in df_gols.select_dtypes(include=['object']).columns:
        df_gols[col] = df_gols[col].apply(normalize_string)
    
    for col in df_cartoes.select_dtypes(include=['object']).columns:
        df_cartoes[col] = df_cartoes[col].apply(normalize_string)
    
    for col in df_estatisticas.select_dtypes(include=['object']).columns:
        df_estatisticas[col] = df_estatisticas[col].apply(normalize_string)
    
    # Converter strings percentuais em estatísticas
    if 'posse_de_bola' in df_estatisticas.columns:
        df_estatisticas['posse_de_bola'] = pd.to_numeric(
            df_estatisticas['posse_de_bola'].astype(str).str.rstrip('%'), 
            errors='coerce'
        )
    
    if 'precisao_passes' in df_estatisticas.columns:
        df_estatisticas['precisao_passes'] = pd.to_numeric(
            df_estatisticas['precisao_passes'].astype(str).str.rstrip('%'), 
            errors='coerce'
        )
    
    print("      Dados normalizados")
    
    #  2. BASE: PARTIDAS 
    print("\n[2/5] Preparando base de partidas...")
    df = df_partidas.copy()
    
    # Padronizar nomes de colunas
    df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
    
    # Converter data
    df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y', errors='coerce')
    
    # Criar colunas derivadas - Data
    df['ano'] = df['data'].dt.year
    df['mes'] = df['data'].dt.month
    df['dia_semana'] = df['data'].dt.day_name()
    
    # Criar colunas derivadas - Resultado
    df['tipo_resultado'] = df.apply(
        lambda row: 'Vitória Mandante' if row['vencedor'] == row['mandante']
        else ('Vitória Visitante' if row['vencedor'] == row['visitante'] else 'Empate'),
        axis=1
    )
    
    # Criar colunas derivadas - Gols
    df['diferenca_gols'] = abs(df['mandante_placar'] - df['visitante_placar'])
    df['total_gols'] = df['mandante_placar'] + df['visitante_placar']
    df['foi_equilibrado'] = df['diferenca_gols'] <= 1
    df['foi_goleada'] = df['diferenca_gols'] >= 3
    
    # Remover colunas com muitos nulos (>50%)
    cols_to_drop = ['formacao_mandante', 'formacao_visitante', 'tecnico_mandante', 'tecnico_visitante']
    df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors='ignore')
    
    print(f"      {len(df):,} partidas processadas")
    
    #  3. AGREGAR: ESTATÍSTICAS 
    print("\n[3/5] Agregando estatísticas...")
    
    df_estatisticas.columns = df_estatisticas.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
    
    stats_cols = ['chutes', 'chutes_no_alvo', 'posse_de_bola', 'passes', 
                  'precisao_passes', 'faltas', 'cartao_amarelo', 'cartao_vermelho', 
                  'impedimentos', 'escanteios']
    
    # MANDANTE
    df_stats_m = df_estatisticas.merge(
        df[['id', 'mandante']], left_on='partida_id', right_on='id', how='inner'
    )
    df_stats_m = df_stats_m[df_stats_m['clube'] == df_stats_m['mandante']]
    for col in stats_cols:
        if col in df_stats_m.columns:
            df_stats_m = df_stats_m.rename(columns={col: f'mandante_{col}'})
    df_stats_m = df_stats_m[['partida_id'] + [f'mandante_{col}' for col in stats_cols if col in df_estatisticas.columns]]
    
    # VISITANTE
    df_stats_v = df_estatisticas.merge(
        df[['id', 'visitante']], left_on='partida_id', right_on='id', how='inner'
    )
    df_stats_v = df_stats_v[df_stats_v['clube'] == df_stats_v['visitante']]
    for col in stats_cols:
        if col in df_stats_v.columns:
            df_stats_v = df_stats_v.rename(columns={col: f'visitante_{col}'})
    df_stats_v = df_stats_v[['partida_id'] + [f'visitante_{col}' for col in stats_cols if col in df_estatisticas.columns]]
    
    # Merge
    df = df.merge(df_stats_m, left_on='id', right_on='partida_id', how='left')
    df = df.merge(df_stats_v, left_on='id', right_on='partida_id', how='left', suffixes=('', '_v'))
    
    print("      Estatísticas agregadas")
    
    #  4. AGREGAR: GOLS 
    print("\n[4/5] Agregando gols...")
    
    df_gols.columns = df_gols.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
    
    gols_agg = df_gols.groupby('partida_id').agg({
        'tipo_de_gol': lambda x: (x == 'Gol Contra').sum(),
        'atleta': 'count'
    }).rename(columns={'tipo_de_gol': 'gols_contra', 'atleta': 'total_gols_registrados'})
    
    gols_agg['gols_penalty'] = df_gols[df_gols['tipo_de_gol'] == 'Penalty'].groupby('partida_id').size()
    gols_agg['gols_penalty'] = gols_agg['gols_penalty'].fillna(0).astype(int)
    
    df = df.merge(gols_agg, left_on='id', right_index=True, how='left')
    df['gols_contra'] = df['gols_contra'].fillna(0).astype(int)
    df['gols_penalty'] = df['gols_penalty'].fillna(0).astype(int)
    
    print("      Gols agregados")
    
    #  5. AGREGAR: CARTÕES 
    print("\n[5/5] Agregando cartões...")
    
    df_cartoes.columns = df_cartoes.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
    
    cartoes_agg = df_cartoes.groupby('partida_id').agg({
        'cartao': lambda x: (x == 'Amarelo').sum(),
        'atleta': 'count'
    }).rename(columns={'cartao': 'total_cartoes_amarelos', 'atleta': 'total_cartoes'})
    
    cartoes_agg['total_cartoes_vermelhos'] = df_cartoes[df_cartoes['cartao'] == 'Vermelho'].groupby('partida_id').size()
    cartoes_agg['total_cartoes_vermelhos'] = cartoes_agg['total_cartoes_vermelhos'].fillna(0).astype(int)
    
    df = df.merge(cartoes_agg, left_on='id', right_index=True, how='left')
    df['total_cartoes_amarelos'] = df['total_cartoes_amarelos'].fillna(0).astype(int)
    df['total_cartoes_vermelhos'] = df['total_cartoes_vermelhos'].fillna(0).astype(int)
    df['total_cartoes'] = df['total_cartoes'].fillna(0).astype(int)
    
    print("      Cartões agregados")
    
    #  6. LIMPEZA FINAL 
    print("\n[6/6] Limpeza final...")
    
    # Remover colunas temporárias
    df = df.drop(columns=['partida_id', 'partida_id_v'], errors='ignore')
    
    # Renomear ID
    df = df.rename(columns={'id': 'partida_id'})
    
    # Tratar valores nulos em estatísticas (preencher com 0 onde faz sentido)
    stat_numeric_cols = [col for col in df.columns if any(x in col for x in ['chutes', 'passes', 'faltas', 'impedimentos', 'escanteios'])]
    for col in stat_numeric_cols:
        if df[col].dtype in ['float64', 'int64']:
            df[col] = df[col].fillna(0)
    
    # Tratar percentuais nulos (preencher com média)
    percent_cols = [col for col in df.columns if 'posse_de_bola' in col or 'precisao_passes' in col]
    for col in percent_cols:
        if col in df.columns and df[col].dtype in ['float64']:
            df[col] = df[col].fillna(df[col].mean())
    
    print("      Dados limpos")
    
    print("\n" + "="*70)
    print("TRANSFORMAÇÃO CONCLUÍDA!")
    print(f"Big Table Silver: {len(df):,} linhas × {len(df.columns)} colunas")
    print("="*70)
    
    return df


print("Função de transformação definida")

Função de transformação definida


## 2. Extract - Carregamento dos Dados Brutos

In [7]:
# Caminhos dos arquivos brutos
csv_path = '../Data Layer/raw/'

print("Carregando dados RAW...\n")

# PARTIDAS
print("[1/4] Carregando partidas...")
df_partidas = pd.read_csv(f'{csv_path}campeonato-brasileiro-full.csv')
print(f"      {len(df_partidas):,} partidas carregadas")

# GOLS
print("[2/4] Carregando gols...")
df_gols = pd.read_csv(f'{csv_path}campeonato-brasileiro-gols.csv')
print(f"      {len(df_gols):,} gols carregados")

# CARTÕES
print("[3/4] Carregando cartões...")
df_cartoes = pd.read_csv(f'{csv_path}campeonato-brasileiro-cartoes.csv')
print(f"      {len(df_cartoes):,} cartões carregados")

# ESTATÍSTICAS
print("[4/4] Carregando estatísticas...")
df_estatisticas = pd.read_csv(f'{csv_path}campeonato-brasileiro-estatisticas-full.csv')
print(f"      {len(df_estatisticas):,} estatísticas carregadas")

print("\nTodos os CSVs carregados com sucesso!")

Carregando dados RAW...

[1/4] Carregando partidas...
      8,785 partidas carregadas
[2/4] Carregando gols...
      9,861 gols carregados
[3/4] Carregando cartões...
      20,953 cartões carregados
[4/4] Carregando estatísticas...
      17,570 estatísticas carregadas

Todos os CSVs carregados com sucesso!


## 3. Transform - Transformação e Limpeza dos Dados

Aplicando transformações para criar a Big Table desnormalizada da camada Silver.

In [8]:
# Aplicar todas as transformações
df_silver = transformar_dados_silver(df_partidas, df_gols, df_cartoes, df_estatisticas)

print(f"\nEstatísticas da transformação:")
print(f"Registros transformados: {len(df_silver):,}")
print(f"Colunas finais: {len(df_silver.columns)}")
print(f"Times únicos: {df_silver['mandante'].nunique()}")
print(f"Período: {df_silver['ano'].min()} - {df_silver['ano'].max()}")


INICIANDO TRANSFORMAÇÃO RAW → SILVER

[1/5] Limpeza e normalização de dados...
      Dados normalizados

[2/5] Preparando base de partidas...
      8,785 partidas processadas

[3/5] Agregando estatísticas...
      Estatísticas agregadas

[4/5] Agregando gols...
      Gols agregados

[5/5] Agregando cartões...
      Cartões agregados

[6/6] Limpeza final...
      Dados limpos

TRANSFORMAÇÃO CONCLUÍDA!
Big Table Silver: 8,785 linhas × 46 colunas

Estatísticas da transformação:
Registros transformados: 8,785
Colunas finais: 46
Times únicos: 45
Período: 2003 - 2024


## 4. Load - Carregamento no PostgreSQL

Carregar os dados transformados no banco de dados PostgreSQL (schema `silver`, tabela `tb_partidas_silver`).

In [9]:
# Testar conexão com banco
print("Testando conexão com PostgreSQL...")
try:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT version();"))
        version = result.fetchone()[0]
        print(f"Conexão estabelecida!")
        print(f"PostgreSQL: {version.split(',')[0]}")
except Exception as e:
    print(f"Erro na conexão: {e}")
    print("Verifique se o Docker está rodando!")

Testando conexão com PostgreSQL...
Conexão estabelecida!
PostgreSQL: PostgreSQL 15.15 on x86_64-pc-linux-musl


In [10]:
# Carregar dados no PostgreSQL
print("\nCarregando dados no PostgreSQL...")
print(f"Schema: silver")
print(f"Tabela: tb_partidas_silver")
print(f"Registros: {len(df_silver):,}\n")

try:
    df_silver.to_sql(
        'tb_partidas_silver', 
        engine, 
        schema='silver',
        if_exists='replace',
        index=False,
        method='multi',
        chunksize=1000
    )
    print("Dados carregados com sucesso!")
except Exception as e:
    print(f"Erro ao carregar dados: {e}")
    print("\nSalvando backup em CSV...")
    df_silver.to_csv('../Data Layer/silver/tb_partidas_silver_backup.csv', index=False)
    print("Backup salvo!")


Carregando dados no PostgreSQL...
Schema: silver
Tabela: tb_partidas_silver
Registros: 8,785

Dados carregados com sucesso!


In [11]:
# Verificar dados carregados
print("\nVerificando dados no banco...")
query = "SELECT COUNT(*) as total FROM silver.tb_partidas_silver"
result = pd.read_sql(query, engine)
print(f"Total de registros no banco: {result['total'][0]:,}")

# Consulta de amostra
print("\nAmostra dos dados (5 primeiras linhas):\n")
query_sample = "SELECT * FROM silver.tb_partidas_silver LIMIT 5"
pd.read_sql(query_sample, engine)


Verificando dados no banco...
Total de registros no banco: 8,785

Amostra dos dados (5 primeiras linhas):



,partida_id,rodata,data,hora,mandante,visitante,vencedor,arena,mandante_placar,visitante_placar,...,visitante_cartao_amarelo,visitante_cartao_vermelho,visitante_impedimentos,visitante_escanteios,gols_contra,total_gols_registrados,gols_penalty,total_cartoes_amarelos,total_cartoes,total_cartoes_vermelhos
0,1,1,2003-03-29,16:00,Guarani,Vasco,Guarani,Brinco de Ouro,4,2,...,0,0,0,0,0,None,0,0,0,0
1,2,1,2003-03-29,16:00,Athletico-PR,Gremio,Athletico-PR,Arena da Baixada,2,0,...,0,0,0,0,0,None,0,0,0,0
2,3,1,2003-03-30,16:00,Flamengo,Coritiba,-,Maracanã,1,1,...,0,0,0,0,0,None,0,0,0,0
3,4,1,2003-03-30,16:00,Goias,Paysandu,-,Serra Dourada,2,2,...,0,0,0,0,0,None,0,0,0,0
4,5,1,2003-03-30,16:00,Internacional,Ponte Preta,-,Beira Rio,1,1,...,0,0,0,0,0,None,0,0,0,0


In [8]:
%pip install sqlalchemy psycopg2-binary

Note: you may need to restart the kernel to use updated packages.
